In [1]:
import os
import pandas as pd
import pdfplumber
import requests
import json
import re
import warnings
warnings.filterwarnings("ignore")
from openai import OpenAI
from typing import List  
  
from tqdm import tqdm  
from langchain_core.prompts import ChatPromptTemplate  
from langchain_core.pydantic_v1 import BaseModel, Field  
from langchain_core.output_parsers import JsonOutputParser  
from langchain_openai import AzureChatOpenAI  
from langchain_core.runnables import RunnableLambda
from langchain_community.document_loaders import UnstructuredFileLoader  
from langchain_text_splitters import RecursiveCharacterTextSplitter

/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/taijieshengwu/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or

In [27]:
# apikey = "sk-mVpKLjvvbUO1CerUNSIRkBKtYaRYdF9anaotBTc164Y6tS9x"
apikey = "sk-slececspLTc947d0am0X3G0y5mwo7qVAofFGTsq3citFJeya"
data_dir = '/Users/taijieshengwu/Documents/相关文本资料汇总'

In [28]:
# # 初始化一个空的 DataFrame
# pdf_data = []

# # 遍历目录下所有 PDF 文件
# for filename in os.listdir(data_dir):
#     if filename.endswith(".pdf"):
#         file_path = os.path.join(data_dir, filename)
#         try:
#             with pdfplumber.open(file_path) as pdf:
#                 text = ''
#                 filename = filename[:-4]
#                 for page in pdf.pages:
#                     text += page.extract_text() or ''
#                 if "_" in filename:
#                     title, author = filename.rsplit("_",1)
#                 else:
#                     title, author = filename, ""

#                 pdf_data.append({
#                     "title": title,
#                     "author": author,
#                     "content": text
#                 })
#         except Exception as e:
#             print(f"读取失败：{filename}, 错误：{e}")

# # 转换为 DataFrame
# df = pd.DataFrame(pdf_data)
# df.to_csv("/Users/taijieshengwu/Documents/arts-RAG/data.csv", index=False, encoding="utf-8-sig")
df = pd.read_csv("/Users/taijieshengwu/Documents/arts-RAG/data.csv", encoding="utf-8-sig")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/taijieshengwu/Documents/arts-RAG/data.csv'

In [29]:
def clean_text_compact(text):
    # 去除 HTML 标签
    text = re.sub(r'<.*?>', '', text)
    # 去除所有空白字符（空格、换行、制表符等）
    text = re.sub(r'\s+', '', text)
    return text

def split_document(text):
    try:  
        cleaned_text = clean_text_compact(text)
        text_splitter = RecursiveCharacterTextSplitter(  
            chunk_size=2048,  
            chunk_overlap=128  
        )  
        documents = text_splitter.split_text(cleaned_text)
        return documents
    except:
        return text

# 示例调用
documents = split_document(df.iloc[0]['content'])
print(documents)
print("段落数：", len(documents))

['艺术教育115艺术管理共同体美学意识的构建———以剧院公共艺术教育策划与管理为例邓玉琪王小龙摘要艺术管理美学作为艺术美学与管理美学的交叉研究，是基于当代艺术主体关系转移、艺术过程要素转变、艺术内容多元转化的前提，在艺术管理视域下提出的美学观念，亦是对艺术美学传统观念的创造性转化与创新性发展。由于艺术管理介入艺术活动，艺术的美学意识发生了多维度的重构，如在主体意识、过程意识、内容意识等层面构建共同体意识。这种共同体意识的构建在剧院管理，尤其在剧院公共艺术教育策划与管理中十分必要。关键词艺术管理;艺术管理美学;剧院公共艺术教育;共同体意识剧院公共艺术教育策划与管理属于剧院管理的和内容，以及城市居民在剧院空间中彼此交流、对研究范畴，剧院管理是艺术管理的分支。从艺术门话的中介;剧院服务作为联结剧院与人、与社会的类上细分，艺术管理可分为音乐管理、戏剧管理、纽带，其公共服务是面向社会公众，是剧院艺术教舞蹈管理、电影管理等;从艺术机构上细分，艺术育活动区别于学校艺术教育的特色所在，并成为剧管理可分为演出现场管理、剧院管理、电影院管理院面向更多社会公众的渠道;剧院文化作为联结剧等。随着艺术管理学科的建设与完善，艺术管理的院与艺术、与人、与社会、与城市的纽带，以教育分类方式会更加丰富。共同体意识在不同领域具有为剧院文化输出的方式，让更多城市居民接受剧院不同的性质，在政治、经济领域属于哲学意识;在文化、体验剧院艺术，以此丰富精神生活。剧院空民族、国家层面属于文化意识;在艺术领域属于美间、剧院艺术、剧院服务、剧院文化是剧院公共艺术教育策划与管理的四大要素，都与人有关，人即学意识。艺术管理美学研究的是艺术管理者以主体主体，包括城市居民即剧院公共艺术教育的受众、身份介入艺术创作者、表演者、接受者的固有体策划与管理者、教育者，同时，剧院空间设计决定系，打破三角形结构，构建菱形结构以后，艺术的着公共艺术教育的实施环境(剧院)，剧院艺术本主体与客体构成多重审美关系、形式与内容构成多体决定着剧院公共教育的实施媒介(艺术)，剧院样审美关系、过程与结果构成多元审美关系，以及服务导向决定着剧院艺术教育的实施特性(公菱形结构中的审美要素等内容。共)，剧院文化理念决定着剧院公共艺术的实施方式(教育)。一、“主体”共同体意识的构建在剧院公共艺术教育策划与管理中，主体因处剧院作为一座城市的文化名片，肩负城市文

In [30]:
QA_PAIRS_SYSTEM_PROMPT = """  
<Context></Context> 你是一个面向公共艺术设计领域的问答系统，标记中是一段关于公共艺术的文献文本，学习和分析它，并整理学习成果：  
- 提出问题并给出每个问题的答案。  
- 答案需详细完整，尽可能保留原文描述。  
- 答案可以包含普通文字、链接、代码、表格、公示、媒体链接等 Markdown 元素。  
- 最多提出 30 个问题。  
"""

QA_PAIRS_HUMAN_PROMPT = """  
请按以下格式整理学习成果:  
<Context>  
文本  
</Context>  
[  
{{"question": "问题1","answer":"答案1"}},  
{{"question": "问题2","answer":"答案2"}},  
]  
------  
  
我们开始吧!  
  
<Context>  
{text}  
<Context/>  
"""

prompt = ChatPromptTemplate.from_messages([  
	("system", QA_PAIRS_SYSTEM_PROMPT),  
	("human", QA_PAIRS_HUMAN_PROMPT)  
])

In [31]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=apikey,
    base_url="https://api.chatanywhere.tech/v1"
)

In [32]:
class QaPair(BaseModel):  
	question: str = Field(description='问题内容')  
	answer: str = Field(description='问题的回答')  
	  
  
class QaPairs(BaseModel):  
	qas: List[QaPair] = Field(description='问答对列表')

parser = JsonOutputParser(pydantic_object=QaPairs)




In [33]:


def chatanywhere_llm(input: dict):
    messages = [
        {"role": "system", "content": QA_PAIRS_SYSTEM_PROMPT},
        {"role": "user", "content": QA_PAIRS_HUMAN_PROMPT.format(text=input['text'])},
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7
    )
    return response.choices[0].message.content

# 包成 Runnable
llm = RunnableLambda(chatanywhere_llm)
parser = JsonOutputParser(pydantic_object=QaPairs)
chain = llm | parser

In [35]:
json_list = []       
non_json_list = []   

for i in range(75,len(df)):
    content = df.iloc[i]['content']
    documents = split_document(content)

    bar = tqdm(total=len(documents), desc=f"Processing row {i}")

    for idx, doc in enumerate(documents):
        bar.update(1)
        try:
            out = chain.invoke({'text': doc})
            for qa in out.qas:
                try:
                    qa_dict = qa.dict() if hasattr(qa, 'dict') else qa
                    json.dumps(qa_dict, ensure_ascii=False)
                    json_list.append(qa_dict)
                    print(qa.dict())
                except Exception as e:
                    non_json_list.append(str(qa))
        except Exception as e:
            print(f"Error invoking chain at row {i}, doc {idx}: {e}")
            non_json_list.append(doc)

print(json_list)
# 追加合法 JSON 到文件
with open('dataset.json', 'a', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=2)
    f.write('\n')  # 可选：写入换行，便于分割多个 JSON 块

# 追加非法 JSON 到另一个文件
with open('non_json_outputs.txt', 'a', encoding='utf-8') as f:
    for line in non_json_list:
        f.write(line + '\n')

Processing row 149: 100%|██████████| 2/2 [02:05<00:00, 62.58s/it]


Processing row 75:  67%|██████▋   | 2/3 [00:02<00:01,  1.03s/it]

Error invoking chain at row 75, doc 0: Error code: 401 - {'error': {'message': 'ApiKey错误(当前请求使用的ApiKey: sk-sle****Jeya)。wrong api key', 'type': 'chatanywhere_error', 'param': None, 'code': '401 UNAUTHORIZED'}}


Processing row 75: 100%|██████████| 3/3 [00:04<00:00,  1.52s/it]

Error invoking chain at row 75, doc 1: Error code: 429 - {'error': {'message': '异常请求过多，暂时禁止访问。Too many requests for exceptions, temporarily disabling access.', 'type': 'chatanywhere_error', 'param': None, 'code': '429 TOO_MANY_REQUESTS'}}
Error invoking chain at row 75, doc 2: Error code: 429 - {'error': {'message': '异常请求过多，暂时禁止访问。Too many requests for exceptions, temporarily disabling access.', 'type': 'chatanywhere_error', 'param': None, 'code': '429 TOO_MANY_REQUESTS'}}


Processing row 75: 100%|██████████| 3/3 [00:07<00:00,  2.41s/it]


Error invoking chain at row 76, doc 0: Error code: 429 - {'error': {'message': '异常请求过多，暂时禁止访问。Too many requests for exceptions, temporarily disabling access.', 'type': 'chatanywhere_error', 'param': None, 'code': '429 TOO_MANY_REQUESTS'}}


KeyboardInterrupt: 